In [9]:
# https://archive.ics.uci.edu/dataset/445/absenteeism+at+work
# Install a pip package in the current Jupyter kernel
# Download the csv file
!python3 -m pip install gdown
!gdown --id 1ppW5XS4eV0-Eq8-DXbdu-QlyTF3h9yd_


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ppW5XS4eV0-Eq8-DXbdu-QlyTF3h9yd_
To: /Users/matthewmartens/Projects/OU-Machine-Learning/Absenteeism_at_work.csv
100%|██████████████████████████████████████| 45.2k/45.2k [00:00<00:00, 2.40MB/s]


In [31]:
#import packages
!python3 -m pip install pandas
!python3 -m pip install numpy
!python3 -m pip install scikit-learn
!python3 -m pip install matplotlib
import pandas as pd #for data-frames
import numpy as np #for matrix operatons
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #split training and test
from sklearn.neighbors import KNeighborsClassifier #knn
from sklearn.metrics import roc_curve #roc
from sklearn.metrics import auc #auc
from sklearn.ensemble import RandomForestClassifier #random forest

In [28]:
#load the data and show the first lines
result = pd.read_csv("absenteeism_at_work.csv", sep=';')
print(f"Total records in csv: {len(result)}")
result.head(10)


Total records in csv: 740


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2
5,3,23,7,6,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
6,10,22,7,6,1,361,52,3,28,239.554,...,0,1,1,1,0,4,80,172,27,8
7,20,23,7,6,1,260,50,11,36,239.554,...,0,1,4,1,0,0,65,168,23,4
8,14,19,7,2,1,155,12,14,34,239.554,...,0,1,2,1,0,0,95,196,25,40
9,1,22,7,2,1,235,11,14,37,239.554,...,0,3,1,0,0,1,88,172,29,8


In [ ]:
np.where(result['Age']=='Accept', 1, 0)

#plt.scatter(x, y)
#plt.show()